# ***Créditos - Distribución***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

## *Data-import*

In [2]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

dataframes = {} # Crear un diccionario para almacenar los DataFrames
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres para obtener solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Imprimir la lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
nombres_df

Lista de DataFrames creados:


['df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_puebla',
 'df_alfa_q_sep_2023_pachuca',
 'df_alfa_q_sep_2023_tulancingo',
 'df_enero_2024_querertaro',
 'df_feb_2024_puebla',
 'df_financiamientos_2019_pachuca',
 'df_financiamientos_2019_puebla',
 'df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2022_puebla',
 'd

___
___
# **1** *Distribución de créditos por segmento*

## **Pachuca**

### 2022

In [6]:
dist_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','monto']]
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2022_pachuca['segmento'] = dist_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1187833169.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_financiamientos_2022_pachuca['segmento'] = dist_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)


,id,monto,segmento
0,1,0.00,E
1,2,1563551.33,D
2,3,1263770.32,D
3,4,3614324.42,A
4,5,252907.18,E


In [7]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2022_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2022_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2022_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2022_pachuca

,Segmento,Cantidad,Porcentaje
5,A,54,3.890490
4,B,85,6.123919
2,C,149,10.734870
1,D,236,17.002882
0,E,758,54.610951
3,S,106,7.636888


In [8]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2022_pachuca['Porcentaje']
labels = creditos_segmentos_2022_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_segmento_2022_pachuca', carpeta='graficas')

fig.show()

### 2023

In [9]:
dist_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','monto']]
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2023_pachuca['segmento'] = dist_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2798921774.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,0.0,E
1,2,0.0,E
2,3,0.0,E
3,4,0.0,E
4,5,0.0,E


In [10]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023_pachuca

,Segmento,Cantidad,Porcentaje
5,A,19,2.790015
3,B,43,6.314244
2,C,63,9.251101
1,D,127,18.649046
0,E,386,56.681351
4,S,43,6.314244


In [11]:
porcentajes = creditos_segmentos_2023_pachuca['Porcentaje']
labels = creditos_segmentos_2023_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2023', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_segmento_2023_pachuca', carpeta='graficas')

fig.show()

## **Tulancingo**

### 2022

In [12]:
dist_financiamientos_2022_tulancingo = df_financiamientos_2022_tulancingo[['id','monto']]
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2022_tulancingo['segmento'] = dist_financiamientos_2022_tulancingo['monto'].apply(asignar_segmento)
dist_financiamientos_2022_tulancingo.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3286797223.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,0.00,E
1,2,0.00,E
2,3,14398.00,E
3,4,96283.00,E
4,5,159720.57,E


In [13]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2022_tulancingo['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2022_tulancingo)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2022_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2022_pachuca

,Segmento,Cantidad,Porcentaje
4,A,2,0.632911
3,B,11,3.481013
2,C,19,6.012658
1,D,65,20.569620
0,E,217,68.670886
5,S,2,0.632911


In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2022_pachuca['Porcentaje']
labels = creditos_segmentos_2022_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_X_segmento_2022_tulgo', carpeta='graficas')

fig.show()

### 2023

In [15]:
dist_financiamientos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','monto']]
# Definir los rangos de monto de cada segmento  'df_financiamientos_2023_tulancingo',
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2023_pachuca['segmento'] = dist_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2403526810.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,1585485.28,D
1,2,608908.05,E
2,3,1918694.86,C
3,4,1241000.00,D
4,5,0.00,E


In [16]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023_pachuca

,Segmento,Cantidad,Porcentaje
3,B,1,2.222222
2,C,7,15.555556
1,D,17,37.777778
0,E,20,44.444444


In [17]:
porcentajes = creditos_segmentos_2023_pachuca['Porcentaje']
labels = creditos_segmentos_2023_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2023', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_X_segmento_2023_tulgo', carpeta='graficas')

fig.show()

## **Puebla**

### 2022

In [18]:
dist_financiamientos_2022_tulancingo = df_financiamientos_2022_puebla[['id','monto']]
# Definir los rangos de monto de cada segmento df_financiamientos_2022_puebla
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2022_tulancingo['segmento'] = dist_financiamientos_2022_tulancingo['monto'].apply(asignar_segmento)
dist_financiamientos_2022_tulancingo.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2960287066.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,21093.00,E
1,2,34318.00,E
2,3,43431.00,E
3,4,51998.00,E
4,5,94648.96,E


In [19]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2022_tulancingo['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2022_tulancingo)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2022_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2022_pachuca

,Segmento,Cantidad,Porcentaje
5,A,112,3.608247
3,B,219,7.055412
2,C,316,10.180412
1,D,546,17.590206
0,E,1705,54.929124
4,S,203,6.539948


In [20]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2022_pachuca['Porcentaje']
labels = creditos_segmentos_2022_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_X_segmento_2022_puebla', carpeta='graficas')

fig.show()

### 2023

In [21]:
dist_financiamientos_2023_pachuca = df_financiamientos_2023_puebla[['id','monto']]
# Definir los rangos de monto de cada segmento  'df_financiamientos_2023_tulancingo',
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2023_pachuca['segmento'] = dist_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\275916748.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,19817.75,E
1,2,0.00,E
2,3,0.00,E
3,4,33908.00,E
4,5,277298.00,E


In [22]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023_pachuca

,Segmento,Cantidad,Porcentaje
5,A,330,3.045123
4,B,544,5.019839
2,C,1023,9.439882
1,D,1800,16.609763
0,E,6481,59.804374
3,S,640,5.905693


In [23]:
porcentajes = creditos_segmentos_2023_pachuca['Porcentaje']
labels = creditos_segmentos_2023_pachuca['Segmento']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2023', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_X_segmento_2023_peubla', carpeta='graficas')

fig.show()

### 2024

In [30]:
dist_financiamientos_2023_pachuca = df_financiamientos_2024_puebla[['id','monto']]
# Definir los rangos de monto de cada segmento  'df_financiamientos_2023_tulancingo',
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2023_pachuca['segmento'] = dist_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3569177752.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,8574319.0,S
1,2,1633800.0,C
2,3,0.0,E
3,4,0.0,E
4,5,0.0,E


In [31]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023_pachuca

,Segmento,Cantidad,Porcentaje
5,A,70,2.803364
4,B,142,5.686824
2,C,230,9.211053
1,D,423,16.940328
0,E,1478,59.191029
3,S,153,6.127353


In [32]:
porcentajes = creditos_segmentos_2023_pachuca['Porcentaje']
labels = creditos_segmentos_2023_pachuca['Segmento']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2023', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditos_X_segmento_2024_peubla', carpeta='graficas')

fig.show()

___
___
# **2** *Distribución % de créditos por segmento (trimestre)*

## **PACHUCA**

In [33]:
trimestre_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','mes','monto']]
trimestre_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','mes','monto']]

### 2022

In [34]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2022_pachuca['trimestre'] = trimestre_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3798387835.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,1563551.33,1
2,3,1,1263770.32,1
3,4,1,3614324.42,1
4,5,1,252907.18,1
...,...,...,...,...
1383,1384,12,0.00,4
1384,1385,12,0.00,4
1385,1386,12,0.00,4
1386,1387,12,0.00,4


In [35]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2022_pachuca['segmento'] = trimestre_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\750050574.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,1563551.33,1,D
2,3,1,1263770.32,1,D
3,4,1,3614324.42,1,A
4,5,1,252907.18,1,E


In [36]:
trimestres_credit_2022_pachuca = trimestre_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,D
2,1,D
3,1,A
4,1,E
...,...,...
1383,4,E
1384,4,E
1385,4,E
1386,4,E


In [37]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1388
trimestre          1          2          3          4
segmento                                             
A           0.576369   0.504323   0.720461   0.720461
B           1.080692   1.512968   1.080692   1.585014
C           2.953890   2.521614   2.233429   2.593660
D           4.610951   5.043228   5.475504   4.682997
E          13.112392  12.103746  12.752161  16.642651
S           1.368876   1.585014   2.305476   2.233429


In [38]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2022_pachuca', carpeta='graficas')

fig.show()


### 2023

In [39]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2023_pachuca['trimestre'] = trimestre_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1639858340.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,0.00,1
4,5,1,0.00,1
...,...,...,...,...
676,677,7,4817952.41,3
677,678,7,5949855.28,3
678,679,7,8000000.00,3
679,680,7,9650850.00,3


In [40]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2023_pachuca['segmento'] = trimestre_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4014434308.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,0.0,1,E
1,2,1,0.0,1,E
2,3,1,0.0,1,E
3,4,1,0.0,1,E
4,5,1,0.0,1,E


In [41]:
trimestres_credit_2023_pachuca = trimestre_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
676,3,S
677,3,S
678,3,S
679,3,S


In [42]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  681
trimestre          1          2         3
segmento                                 
A           0.881057   1.027900  0.146843
B           1.762115   1.908957  1.468429
C           4.552129   2.790015  1.027900
D           7.929515   9.397944  4.258443
E          21.292217  28.046990  7.342144
S           2.936858   2.349486  0.881057


In [43]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2023_pachuca', carpeta='graficas')

fig.show()


## **Tulancingo**

In [45]:
trimestre_financiamientos_2019_pachuca = df_financiamientos_2019_tulancingo[['id','mes','monto']]
trimestre_financiamientos_2020_pachuca = df_financiamientos_2020_tulancingo[['id','mes','monto']]
trimestre_financiamientos_2021_pachuca = df_financiamientos_2021_tulancingo[['id','mes','monto']]
trimestre_financiamientos_2022_pachuca = df_financiamientos_2022_tulancingo[['id','mes','monto']]
trimestre_financiamientos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','mes','monto']]

### 2022

In [46]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2022_pachuca['trimestre'] = trimestre_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3798387835.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,0.00,1
2,3,1,14398.00,1
3,4,1,96283.00,1
4,5,1,159720.57,1
...,...,...,...,...
311,312,12,2279250.00,4
312,313,12,2285795.72,4
313,314,12,2661350.00,4
314,315,12,2946484.00,4


In [47]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2022_pachuca['segmento'] = trimestre_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\750050574.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,0.00,1,E
2,3,1,14398.00,1,E
3,4,1,96283.00,1,E
4,5,1,159720.57,1,E


In [48]:
trimestres_credit_2022_pachuca = trimestre_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
311,4,C
312,4,C
313,4,B
314,4,B


In [49]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  316
trimestre          1          2          3          4
segmento                                             
A           0.316456   0.000000   0.316456   0.000000
B           0.632911   0.949367   0.316456   0.632911
C           1.265823   1.265823   0.949367   2.531646
D           4.430380   5.379747   7.278481   4.430380
E          18.037975  12.658228  19.936709  18.037975
S           0.000000   0.316456   0.000000   0.316456


In [50]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2022_tulgo', carpeta='graficas')

fig.show()


### 2023

In [51]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2023_pachuca['trimestre'] = trimestre_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1639858340.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,1585485.28,1
1,2,1,608908.05,1
2,3,1,1918694.86,1
3,4,1,1241000.00,1
4,5,2,0.00,1
5,6,2,390087.63,1
6,7,2,604900.00,1
7,8,2,1429200.00,1
8,9,2,1900000.00,1
9,10,2,1100000.00,1


In [52]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2023_pachuca['segmento'] = trimestre_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4014434308.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,1585485.28,1,D
1,2,1,608908.05,1,E
2,3,1,1918694.86,1,C
3,4,1,1241000.00,1,D
4,5,2,0.00,1,E


In [53]:
trimestres_credit_2023_pachuca = trimestre_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,D
1,1,E
2,1,C
3,1,D
4,1,E
5,1,E
6,1,E
7,1,D
8,1,C
9,1,D


In [54]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  45
trimestre          1          2          3
segmento                                  
B           2.222222   0.000000   0.000000
C           4.444444   2.222222   6.666667
D          13.333333  15.555556  11.111111
E          20.000000  11.111111  13.333333


In [55]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2023_tulgo', carpeta='graficas')

fig.show()


## **Puebla**

In [75]:
trimestre_financiamientos_2022_pachuca = df_financiamientos_2022_puebla[['id','mes','monto']]
trimestre_financiamientos_2023_pachuca = df_financiamientos_2023_puebla[['id','mes','monto']]
trimestre_financiamientos_2024_pachuca = df_financiamientos_2024_puebla[['id','mes','monto']]

### 2022

In [60]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2022_pachuca['trimestre'] = trimestre_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3798387835.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,21093.00,1
1,2,1,34318.00,1
2,3,1,43431.00,1
3,4,1,51998.00,1
4,5,1,94648.96,1
...,...,...,...,...
3099,3100,12,0.00,4
3100,3101,12,0.00,4
3101,3102,12,0.00,4
3102,3103,12,0.00,4


In [61]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2022_pachuca['segmento'] = trimestre_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\750050574.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,21093.00,1,E
1,2,1,34318.00,1,E
2,3,1,43431.00,1,E
3,4,1,51998.00,1,E
4,5,1,94648.96,1,E


In [62]:
trimestres_credit_2022_pachuca = trimestre_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
3099,4,E
3100,4,E
3101,4,E
3102,4,E


In [63]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  3104
trimestre          1          2          3          4
segmento                                             
A           0.579897   0.966495   0.740979   0.676546
B           1.192010   1.417526   1.288660   1.643041
C           2.351804   2.641753   2.384021   2.480670
D           5.315722   5.025773   5.122423   5.122423
E          15.077320  13.434278  14.143041  12.274485
S           1.030928   1.739691   1.643041   1.707474


In [64]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2022_puebla', carpeta='graficas')

fig.show()


### 2023

In [65]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2023_pachuca['trimestre'] = trimestre_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1639858340.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,19817.75,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,33908.00,1
4,5,1,277298.00,1
...,...,...,...,...
10832,10833,12,67786.64,4
10833,10834,12,0.00,4
10834,10835,12,0.00,4
10835,10836,12,0.00,4


In [66]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2023_pachuca['segmento'] = trimestre_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4014434308.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,19817.75,1,E
1,2,1,0.00,1,E
2,3,1,0.00,1,E
3,4,1,33908.00,1,E
4,5,1,277298.00,1,E


In [67]:
trimestres_credit_2023_pachuca = trimestre_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
10832,4,E
10833,4,E
10834,4,E
10835,4,E


In [68]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  10837
trimestre          1          2          3          4
segmento                                             
A           0.424472   0.590569   0.525976   0.756667
B           0.904309   0.941220   1.153456   1.245732
C           1.993172   2.030082   2.186952   2.371505
D           4.189351   4.881425   4.992157   5.213620
E          13.869152  15.364031  13.795331  16.775860
S           1.144228   1.476423   1.504106   1.670204


In [69]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2023_puebla', carpeta='graficas')

fig.show()


### 2024

In [76]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2024_pachuca['trimestre'] = trimestre_financiamientos_2024_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2024_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4174566939.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,8574319.0,1
1,2,1,1633800.0,1
2,3,1,0.0,1
3,4,1,0.0,1
4,5,1,0.0,1
...,...,...,...,...
2492,2493,3,0.0,1
2493,2494,3,0.0,1
2494,2495,3,0.0,1
2495,2496,3,0.0,1


In [77]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2024_pachuca['segmento'] = trimestre_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2024_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2371138286.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,8574319.0,1,S
1,2,1,1633800.0,1,D
2,3,1,0.0,1,E
3,4,1,0.0,1,E
4,5,1,0.0,1,E


In [79]:
trimestres_credit_2023_pachuca = trimestre_financiamientos_2024_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,S
1,1,D
2,1,E
3,1,E
4,1,E
...,...,...
2492,1,E
2493,1,E
2494,1,E
2495,1,E


In [80]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  2497
trimestre          1
segmento            
A           2.442932
B           4.245094
C           9.050861
D          19.383260
E          59.191029
S           5.686824


In [81]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_segmento_trimestre2024_puebla', carpeta='graficas')

fig.show()


# tog

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f' {trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    title='Distribución de créditos por segmento (trimestre) 2023',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
#Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditosXsegmento_trimestre2023_pachuca', carpeta='graficas')

fig.show()


___
___
# **3** *Distribución de créditos por monto*

## **Pachuca**

In [84]:
monto_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','monto']]
monto_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','monto']]

In [85]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2022_pachuca['rango'] = monto_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4270173908.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.00,1
1,2,1563551.33,2
2,3,1263770.32,2
3,4,3614324.42,4
4,5,252907.18,1


In [86]:
conteo_por_rango = monto_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,758
1,2,333
2,3,137
3,5,104
4,4,56


In [87]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,758
1,2,333
2,3,137
4,4,56
3,5,104


In [88]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,758
1,< 2M,333
2,< 3M,137
4,< 4M,56
3,< 20M,104


In [89]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2022_pachuca', carpeta='graficas')

fig.show()


In [90]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2023_pachuca['rango'] = monto_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1660705631.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.0,1
1,2,0.0,1
2,3,0.0,1
3,4,0.0,1
4,5,0.0,1


In [91]:
conteo_por_rango = monto_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,386
1,2,167
2,3,66
4,4,20
3,5,42


In [92]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Definir un diccionario de mapeo con el orden deseado de las etiquetas
mapeo_ordenado = {
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
}

# Reemplazar los valores de rango con las etiquetas más cortas y descriptivas, manteniendo el orden
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace(mapeo_ordenado.keys(), mapeo_ordenado.values())
df_conteo_por_rango

,rango,conteo
0,< 1M,386
1,< 2M,167
2,< 3M,66
4,< 4M,20
3,< 20M,42


In [93]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2023',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2023_pachuca', carpeta='graficas')

fig.show()


## **Tulancingo**

In [94]:
monto_financiamientos_2022_pachuca = df_financiamientos_2022_tulancingo[['id','monto']]
monto_financiamientos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','monto']]

In [95]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2022_pachuca['rango'] = monto_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4270173908.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.00,1
1,2,0.00,1
2,3,14398.00,1
3,4,96283.00,1
4,5,159720.57,1


In [96]:
conteo_por_rango = monto_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,217
1,2,77
2,3,18
3,4,2
4,5,2


In [97]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,217
1,2,77
2,3,18
3,4,2
4,5,2


In [98]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,217
1,< 2M,77
2,< 3M,18
3,< 4M,2
4,< 20M,2


In [99]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2022_tulgo', carpeta='graficas')

fig.show()


In [100]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2023_pachuca['rango'] = monto_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1660705631.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,1585485.28,2
1,2,608908.05,1
2,3,1918694.86,2
3,4,1241000.00,2
4,5,0.00,1


In [101]:
conteo_por_rango = monto_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
1,1,20
0,2,22
2,3,3


In [102]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Definir un diccionario de mapeo con el orden deseado de las etiquetas
mapeo_ordenado = {
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
}

# Reemplazar los valores de rango con las etiquetas más cortas y descriptivas, manteniendo el orden
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace(mapeo_ordenado.keys(), mapeo_ordenado.values())
df_conteo_por_rango

,rango,conteo
1,< 1M,20
0,< 2M,22
2,< 3M,3


In [103]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2023',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2023_tulgo', carpeta='graficas')

fig.show()


## **Puebla**

In [121]:
monto_financiamientos_2022_pachuca = df_financiamientos_2022_puebla[['id','monto']]
monto_financiamientos_2023_pachuca = df_financiamientos_2023_puebla[['id','monto']]
monto_financiamientos_2024_pachuca = df_financiamientos_2024_puebla[['id','monto']]

### 2022

In [105]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2022_pachuca['rango'] = monto_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4270173908.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,21093.00,1
1,2,34318.00,1
2,3,43431.00,1
3,4,51998.00,1
4,5,94648.96,1


In [106]:
conteo_por_rango = monto_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,1705
1,2,747
2,3,334
3,5,190
4,4,128


In [107]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,1705
1,2,747
2,3,334
4,4,128
3,5,190


In [108]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,1705
1,< 2M,747
2,< 3M,334
4,< 4M,128
3,< 20M,190


In [109]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2022_puebla', carpeta='graficas')

fig.show()


### 2023

In [110]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2023_pachuca['rango'] = monto_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1660705631.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,19817.75,1
1,2,0.00,1
2,3,0.00,1
3,4,33908.00,1
4,5,277298.00,1


In [111]:
conteo_por_rango = monto_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,6481
1,2,2488
2,3,879
4,4,361
3,5,628


In [112]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Definir un diccionario de mapeo con el orden deseado de las etiquetas
mapeo_ordenado = {
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
}

# Reemplazar los valores de rango con las etiquetas más cortas y descriptivas, manteniendo el orden
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace(mapeo_ordenado.keys(), mapeo_ordenado.values())
df_conteo_por_rango

,rango,conteo
0,< 1M,6481
1,< 2M,2488
2,< 3M,879
4,< 4M,361
3,< 20M,628


In [113]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2023',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2023_puebla', carpeta='graficas')

fig.show()


### 2024

In [122]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2024_pachuca['rango'] = monto_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2024_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\3367583840.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,8574319.0,5
1,2,1633800.0,2
2,3,0.0,1
3,4,0.0,1
4,5,0.0,1


In [123]:
conteo_por_rango = monto_financiamientos_2024_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,1478
1,2,572
2,3,223
4,4,82
3,5,142


In [124]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Definir un diccionario de mapeo con el orden deseado de las etiquetas
mapeo_ordenado = {
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
}

# Reemplazar los valores de rango con las etiquetas más cortas y descriptivas, manteniendo el orden
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace(mapeo_ordenado.keys(), mapeo_ordenado.values())
df_conteo_por_rango

,rango,conteo
0,< 1M,1478
1,< 2M,572
2,< 3M,223
4,< 4M,82
3,< 20M,142


In [125]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2023',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditos_X_monto_2024_puebla', carpeta='graficas')

fig.show()


___
___
# **4** *Promedio de valor de Créditos para casas nueva y para vivienda usada*

## **PACHUCA**

### 2023

In [126]:
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','mes','modalidad','monto']]
shape_2023 = finance_2023_pachuca.shape
print(shape_2023)
print(finance_2023_pachuca.head())

(681, 4)
   id  mes  modalidad  monto
0   1    1          1    0.0
1   2    1          2    0.0
2   3    1          2    0.0
3   4    1          2    0.0
4   5    1          2    0.0


In [127]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2023_pachuca['modalidad'] = finance_2023_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2023_pachuca.loc[~finance_2023_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2023_pachuca = finance_2023_pachuca[finance_2023_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2023_pachuca['trimestre'] = (finance_2023_pachuca['mes'] - 1) // 3 + 1
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2932962646.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
13,14,1,Nueva,28311.21,1
21,22,1,Nueva,213779.25,1
23,24,1,Usada,394000.00,1
24,25,1,Nueva,420000.00,1
...,...,...,...,...,...
676,677,7,Nueva,4817952.41,3
677,678,7,Nueva,5949855.28,3
678,679,7,Nueva,8000000.00,3
679,680,7,Nueva,9650850.00,3


In [128]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2023_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023
0,Nueva,"1,910,504.78","1,650,164.14","2,142,282.78"
1,Usada,"1,704,980.61","1,286,440.54","1,397,343.14"


In [129]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,910,504.78"
4,Nueva,2T 2023,"1,650,164.14"
6,Nueva,3T 2023,"2,142,282.78"
3,Usada,1T 2023,"1,704,980.61"
5,Usada,2T 2023,"1,286,440.54"
7,Usada,3T 2023,"1,397,343.14"


In [130]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        text=df_modalidad['promedio'],  # Texto con el valor de promedio
        textposition="top center",  # Posición del texto
    ))
    
fig.update_layout(
    title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pachuca', carpeta='graficas')

fig.show()

In [139]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pachuca', carpeta='graficas')
fig.show()


### 2022

In [140]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','mes','modalidad','monto']]
shape_2022 = finance_2022_pachuca.shape
print(shape_2022)
print(finance_2022_pachuca.head())

(1388, 4)
   id  mes  modalidad       monto
0   1    1          1        0.00
1   2    1          1  1563551.33
2   3    1          1  1263770.32
3   4    1          1  3614324.42
4   5    1          1   252907.18


In [141]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022_pachuca['modalidad'] = finance_2022_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022_pachuca.loc[~finance_2022_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022_pachuca['trimestre'] = (finance_2022_pachuca['mes'] - 1) // 3 + 1
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\284171248.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,1563551.33,1
2,3,1,Nueva,1263770.32,1
3,4,1,Nueva,3614324.42,1
4,5,1,Nueva,252907.18,1
...,...,...,...,...,...
1378,1379,12,Usada,896000.00,4
1379,1380,12,Usada,843983.81,4
1380,1381,12,Usada,1089000.00,4
1381,1382,12,Usada,671476.15,4


In [142]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2022',
    2: '2T 2022',
    3: '3T 2022',
    4: '4T 2022'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2022,2T 2022,3T 2022,4T 2022
0,Nueva,"1,571,750.75","1,996,434.19","2,105,881.09","2,044,172.30"
1,Usada,"1,612,455.21","1,464,666.36","1,668,186.66","1,724,341.95"


In [143]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2022,"1,571,750.75"
4,Nueva,2T 2022,"1,996,434.19"
6,Nueva,3T 2022,"2,105,881.09"
8,Nueva,4T 2022,"2,044,172.30"
3,Usada,1T 2022,"1,612,455.21"
5,Usada,2T 2022,"1,464,666.36"
7,Usada,3T 2022,"1,668,186.66"
9,Usada,4T 2022,"1,724,341.95"


In [149]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]
colors = ['#9500ff', '#ff0059']

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2022)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2022_pachuca', carpeta='graficas')
fig.show()


### 2021

In [150]:
finance_2021_pachuca = df_financiamientos_2021_pachuca[['id','mes','modalidad','monto']]
shape_2021 = finance_2021_pachuca.shape
print(shape_2021)
print(finance_2021_pachuca.head())

(1430, 4)
   id  mes  modalidad     monto
0   1    1          2  20568.00
1   2    1          1  36070.86
2   3    1          2  38824.00
3   4    1          1  39449.85
4   5    1          2  63745.00


In [151]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2021_pachuca['modalidad'] = finance_2021_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2021_pachuca.loc[~finance_2021_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2021_pachuca = finance_2021_pachuca[finance_2021_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2021_pachuca['trimestre'] = (finance_2021_pachuca['mes'] - 1) // 3 + 1
finance_2021_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1979572604.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
1,2,1,Nueva,36070.86,1
3,4,1,Nueva,39449.85,1
5,6,1,Usada,232000.90,1
6,7,1,Nueva,248284.47,1
7,8,1,Nueva,268813.82,1
...,...,...,...,...,...
1425,1426,12,Nueva,7640000.00,4
1426,1427,12,Usada,11719600.00,4
1427,1428,12,Nueva,13490590.00,4
1428,1429,12,Nueva,14009400.00,4


In [152]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_21 = finance_2021_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_21

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,726,001.29","1,640,809.94","1,776,954.58","1,856,235.46"
1,Usada,"1,142,443.24","1,135,746.00","1,229,093.93","1,186,562.79"


In [153]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_21

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,726,001.29"
4,Nueva,2T 2021,"1,640,809.94"
6,Nueva,3T 2021,"1,776,954.58"
8,Nueva,4T 2021,"1,856,235.46"
3,Usada,1T 2021,"1,142,443.24"
5,Usada,2T 2021,"1,135,746.00"
7,Usada,3T 2021,"1,229,093.93"
9,Usada,4T 2021,"1,186,562.79"


In [154]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_21['trimestre']
modalidades = promedio_monto_por_modalidad_mes_21['modalidad']
promedios = promedio_monto_por_modalidad_mes_21['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        text=df_modalidad['promedio'],  # Texto con el valor de promedio
        textposition="top center",  # Posición del texto
    ))
    
fig.update_layout(
    title='Promedio de valor de créditos para viviendas nuevas y usadas (2021)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto')
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcreditnewold_2021_pachuca', carpeta='graficas')

fig.show()

In [156]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_21['trimestre']
modalidades = promedio_monto_por_modalidad_mes_21['modalidad']
promedios = promedio_monto_por_modalidad_mes_21['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        line=dict(color=colors[i % len(colors)]),  # Asignar colores a las líneas
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2021)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2021_pachuca', carpeta='graficas')
fig.show()


### 2020

In [157]:
finance_2020_pachuca = df_financiamientos_2020_pachuca[['id','mes','modalidad','monto']]
shape_2020 = finance_2020_pachuca.shape
print(shape_2020)
print(finance_2020_pachuca.head())

(1178, 4)
   id  mes  modalidad    monto
0   1    1          2  10286.0
1   2    1          2  16471.0
2   3    1          2  25171.0
3   4    1          2  29856.0
4   5    1          2  46811.0


In [158]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2020_pachuca['modalidad'] = finance_2020_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2020_pachuca.loc[~finance_2020_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2020_pachuca = finance_2020_pachuca[finance_2020_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2020_pachuca['trimestre'] = (finance_2020_pachuca['mes'] - 1) // 3 + 1
finance_2020_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2090618855.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
7,8,1,Usada,82200.03,1
9,10,1,Usada,117351.43,1
10,11,1,Usada,126176.36,1
11,12,1,Nueva,187499.00,1
12,13,1,Nueva,200998.49,1
...,...,...,...,...,...
1173,1174,12,Nueva,6099374.96,4
1174,1175,12,Nueva,6584000.00,4
1175,1176,12,Nueva,8904106.00,4
1176,1177,12,Nueva,13390156.00,4


In [159]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_20 = finance_2020_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_20

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,696,609.33","1,658,233.91","1,574,783.97","1,933,412.59"
1,Usada,"884,029.79","872,469.78","1,145,509.06","1,276,401.65"


In [160]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_20

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,696,609.33"
4,Nueva,2T 2021,"1,658,233.91"
6,Nueva,3T 2021,"1,574,783.97"
8,Nueva,4T 2021,"1,933,412.59"
3,Usada,1T 2021,"884,029.79"
5,Usada,2T 2021,"872,469.78"
7,Usada,3T 2021,"1,145,509.06"
9,Usada,4T 2021,"1,276,401.65"


In [161]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_20['trimestre']
modalidades = promedio_monto_por_modalidad_mes_20['modalidad']
promedios = promedio_monto_por_modalidad_mes_20['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2020)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2020_pachuca', carpeta='graficas')
fig.show()


### 2019

In [162]:
finance_2019_pachuca = df_financiamientos_2019_pachuca[['id','mes','modalidad','monto']]
shape_2019 = finance_2019_pachuca.shape
print(shape_2019)
print(finance_2019_pachuca.head())

(1215, 4)
   id  mes  modalidad      monto
0   1    1          2   15000.00
1   2    1          2   31608.00
2   3    1          2   36018.00
3   4    1          1  130000.00
4   5    1          1  134542.22


In [163]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2019_pachuca['modalidad'] = finance_2019_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2019_pachuca.loc[~finance_2019_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2019_pachuca = finance_2019_pachuca[finance_2019_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2019_pachuca['trimestre'] = (finance_2019_pachuca['mes'] - 1) // 3 + 1
finance_2019_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2382405646.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
3,4,1,Nueva,130000.00,1
4,5,1,Nueva,134542.22,1
5,6,1,Usada,140009.61,1
6,7,1,Nueva,152814.52,1
7,8,1,Nueva,186442.16,1
...,...,...,...,...,...
1210,1211,12,Usada,4840750.00,4
1211,1212,12,Nueva,6929939.00,4
1212,1213,12,Nueva,7488269.00,4
1213,1214,12,Nueva,11036275.00,4


In [164]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_19 = finance_2019_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_19

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,519,083.33","1,651,044.95","1,737,192.83","1,724,591.62"
1,Usada,"944,823.31","785,179.29","872,617.81","791,392.54"


In [165]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_19

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,519,083.33"
4,Nueva,2T 2021,"1,651,044.95"
6,Nueva,3T 2021,"1,737,192.83"
8,Nueva,4T 2021,"1,724,591.62"
3,Usada,1T 2021,"944,823.31"
5,Usada,2T 2021,"785,179.29"
7,Usada,3T 2021,"872,617.81"
9,Usada,4T 2021,"791,392.54"


In [166]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_19['trimestre']
modalidades = promedio_monto_por_modalidad_mes_19['modalidad']
promedios = promedio_monto_por_modalidad_mes_19['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2019)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2019_pachuca', carpeta='graficas')
fig.show()


## **TULANCINGO**

### 2023

In [167]:
finance_2023_pachuca = df_financiamientos_2023_tulancingo[['id','mes','modalidad','monto']]
shape_2023 = finance_2023_pachuca.shape
print(shape_2023)
print(finance_2023_pachuca.head())

(45, 4)
   id  mes  modalidad       monto
0   1    1          1  1585485.28
1   2    1          1   608908.05
2   3    1          1  1918694.86
3   4    1          1  1241000.00
4   5    2          1        0.00


In [168]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2023_pachuca['modalidad'] = finance_2023_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2023_pachuca.loc[~finance_2023_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2023_pachuca = finance_2023_pachuca[finance_2023_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2023_pachuca['trimestre'] = (finance_2023_pachuca['mes'] - 1) // 3 + 1
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2932962646.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,1585485.28,1
1,2,1,Nueva,608908.05,1
2,3,1,Nueva,1918694.86,1
3,4,1,Nueva,1241000.00,1
4,5,2,Nueva,0.00,1
5,6,2,Nueva,390087.63,1
6,7,2,Nueva,604900.00,1
7,8,2,Nueva,1429200.00,1
8,9,2,Nueva,1900000.00,1
9,10,2,Nueva,1100000.00,1


In [169]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2023_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023
0,Nueva,"1,072,762.53","1,031,834.45","1,092,332.23"


In [170]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
1,Nueva,1T 2023,"1,072,762.53"
2,Nueva,2T 2023,"1,031,834.45"
3,Nueva,3T 2023,"1,092,332.23"


In [171]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        text=df_modalidad['promedio'],  # Texto con el valor de promedio
        textposition="top center",  # Posición del texto
    ))
    
fig.update_layout(
    title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_tulgo', carpeta='graficas')

fig.show()

In [172]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff','#ff0059']
fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_tulgo', carpeta='graficas')
fig.show()


In [ ]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pachuca', carpeta='graficas')
fig.show()


### 2022

In [173]:
finance_2022_pachuca = df_financiamientos_2022_tulancingo[['id','mes','modalidad','monto']]
shape_2022 = finance_2022_pachuca.shape
print(shape_2022)
print(finance_2022_pachuca.head())

(316, 4)
   id  mes  modalidad      monto
0   1    1          2       0.00
1   2    1          2       0.00
2   3    1          2   14398.00
3   4    1          4   96283.00
4   5    1          3  159720.57


In [174]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022_pachuca['modalidad'] = finance_2022_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022_pachuca.loc[~finance_2022_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022_pachuca['trimestre'] = (finance_2022_pachuca['mes'] - 1) // 3 + 1
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\284171248.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
4,5,1,Usada,159720.57,1
5,6,1,Nueva,241963.35,1
7,8,1,Nueva,369616.07,1
8,9,1,Usada,388575.19,1
9,10,1,Nueva,572183.25,1
...,...,...,...,...,...
311,312,12,Usada,2279250.00,4
312,313,12,Usada,2285795.72,4
313,314,12,Nueva,2661350.00,4
314,315,12,Usada,2946484.00,4


In [175]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2022',
    2: '2T 2022',
    3: '3T 2022',
    4: '4T 2022'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2022,2T 2022,3T 2022,4T 2022
0,Nueva,"1,026,008.53","1,071,404.65","896,403.19","943,715.36"
1,Usada,"754,953.58","1,088,308.40","931,491.31","1,068,239.56"


In [176]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2022,"1,026,008.53"
4,Nueva,2T 2022,"1,071,404.65"
6,Nueva,3T 2022,"896,403.19"
8,Nueva,4T 2022,"943,715.36"
3,Usada,1T 2022,"754,953.58"
5,Usada,2T 2022,"1,088,308.40"
7,Usada,3T 2022,"931,491.31"
9,Usada,4T 2022,"1,068,239.56"


In [177]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]
colors = ['#9500ff', '#ff0059']

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2022)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2022_tulgo', carpeta='graficas')
fig.show()


### 2021

In [178]:
finance_2021_pachuca = df_financiamientos_2021_tulancingo[['id','mes','modalidad','monto']]
shape_2021 = finance_2021_pachuca.shape
print(shape_2021)
print(finance_2021_pachuca.head())

(371, 4)
   id  mes  modalidad       monto
0   1    1          1  1657038.36
1   2    1          2        0.00
2   3    1          2        0.00
3   4    1          3   425823.93
4   5    1          3   792721.29


In [179]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2021_pachuca['modalidad'] = finance_2021_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2021_pachuca.loc[~finance_2021_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2021_pachuca = finance_2021_pachuca[finance_2021_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2021_pachuca['trimestre'] = (finance_2021_pachuca['mes'] - 1) // 3 + 1
finance_2021_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1979572604.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,1657038.36,1
3,4,1,Usada,425823.93,1
4,5,1,Usada,792721.29,1
5,6,1,Usada,495836.57,1
6,7,1,Nueva,806607.00,1
...,...,...,...,...,...
363,364,12,Usada,591186.05,4
364,365,12,Usada,454909.11,4
365,366,12,Usada,1848688.67,4
366,367,12,Usada,1112021.29,4


In [180]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_21 = finance_2021_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_21

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,334,329.81","839,660.96","862,540.25","1,204,522.91"
1,Usada,"786,475.08","862,271.31","817,935.66","934,381.11"


In [181]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_21

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,334,329.81"
4,Nueva,2T 2021,"839,660.96"
6,Nueva,3T 2021,"862,540.25"
8,Nueva,4T 2021,"1,204,522.91"
3,Usada,1T 2021,"786,475.08"
5,Usada,2T 2021,"862,271.31"
7,Usada,3T 2021,"817,935.66"
9,Usada,4T 2021,"934,381.11"


In [182]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_21['trimestre']
modalidades = promedio_monto_por_modalidad_mes_21['modalidad']
promedios = promedio_monto_por_modalidad_mes_21['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        line=dict(color=colors[i % len(colors)]),  # Asignar colores a las líneas
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2021)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2021_tulgo', carpeta='graficas')
fig.show()


### 2020

In [183]:
finance_2020_pachuca = df_financiamientos_2020_tulancingo[['id','mes','modalidad','monto']]
shape_2020 = finance_2020_pachuca.shape
print(shape_2020)
print(finance_2020_pachuca.head())

(389, 4)
   id  mes  modalidad      monto
0   1    1          1  438265.22
1   2    1          1  735918.76
2   3    1          1  312565.61
3   4    1          1  699821.88
4   5    1          2       0.00


In [184]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2020_pachuca['modalidad'] = finance_2020_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2020_pachuca.loc[~finance_2020_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2020_pachuca = finance_2020_pachuca[finance_2020_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2020_pachuca['trimestre'] = (finance_2020_pachuca['mes'] - 1) // 3 + 1
finance_2020_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2090618855.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,438265.22,1
1,2,1,Nueva,735918.76,1
2,3,1,Nueva,312565.61,1
3,4,1,Nueva,699821.88,1
6,7,1,Usada,412563.00,1
...,...,...,...,...,...
380,381,12,Usada,406180.41,4
381,382,12,Usada,514862.04,4
382,383,12,Usada,645675.57,4
383,384,12,Usada,440000.00,4


In [185]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_20 = finance_2020_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_20

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"882,071.66","771,552.09","1,457,134.04","1,381,255.09"
1,Usada,"727,274.65","858,603.06","734,457.82","796,966.33"


In [186]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_20

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"882,071.66"
4,Nueva,2T 2021,"771,552.09"
6,Nueva,3T 2021,"1,457,134.04"
8,Nueva,4T 2021,"1,381,255.09"
3,Usada,1T 2021,"727,274.65"
5,Usada,2T 2021,"858,603.06"
7,Usada,3T 2021,"734,457.82"
9,Usada,4T 2021,"796,966.33"


In [187]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_20['trimestre']
modalidades = promedio_monto_por_modalidad_mes_20['modalidad']
promedios = promedio_monto_por_modalidad_mes_20['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2020)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2020_tulgo', carpeta='graficas')
fig.show()


### 2019

In [188]:
finance_2019_pachuca = df_financiamientos_2019_tulancingo[['id','mes','modalidad','monto']]
shape_2019 = finance_2019_pachuca.shape
print(shape_2019)
print(finance_2019_pachuca.head())

(406, 4)
   id  mes  modalidad       monto
0   1    1          1   369484.31
1   2    1          1  1019902.35
2   3    1          1        0.00
3   4    1          1   675639.85
4   5    1          1  1642474.27


In [189]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2019_pachuca['modalidad'] = finance_2019_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2019_pachuca.loc[~finance_2019_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2019_pachuca = finance_2019_pachuca[finance_2019_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2019_pachuca['trimestre'] = (finance_2019_pachuca['mes'] - 1) // 3 + 1
finance_2019_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2382405646.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,369484.31,1
1,2,1,Nueva,1019902.35,1
2,3,1,Nueva,0.00,1
3,4,1,Nueva,675639.85,1
4,5,1,Nueva,1642474.27,1
...,...,...,...,...,...
391,392,12,Nueva,0.00,4
392,393,12,Nueva,0.00,4
403,404,12,Usada,1231.45,4
404,405,12,Usada,866475.68,4


In [190]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_19 = finance_2019_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_19

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"840,138.08","840,949.48","749,848.13","869,699.34"
1,Usada,"697,168.73","665,900.88","879,353.48","764,900.49"


In [191]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_19

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"840,138.08"
4,Nueva,2T 2021,"840,949.48"
6,Nueva,3T 2021,"749,848.13"
8,Nueva,4T 2021,"869,699.34"
3,Usada,1T 2021,"697,168.73"
5,Usada,2T 2021,"665,900.88"
7,Usada,3T 2021,"879,353.48"
9,Usada,4T 2021,"764,900.49"


In [192]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_19['trimestre']
modalidades = promedio_monto_por_modalidad_mes_19['modalidad']
promedios = promedio_monto_por_modalidad_mes_19['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2019)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2019_tulgo', carpeta='graficas')
fig.show()


## **PUEBLA**

### 2024

In [193]:
finance_2023_pachuca = df_financiamientos_2024_puebla[['id','mes','modalidad','monto']]
shape_2023 = finance_2023_pachuca.shape
print(shape_2023)
print(finance_2023_pachuca.head())

(2497, 4)
   id  mes  modalidad      monto
0   1    1          2  8574319.0
1   2    1          4  1633800.0
2   3    1          2        0.0
3   4    1          2        0.0
4   5    1          2        0.0


In [194]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2023_pachuca['modalidad'] = finance_2023_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2023_pachuca.loc[~finance_2023_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2023_pachuca = finance_2023_pachuca[finance_2023_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2023_pachuca['trimestre'] = (finance_2023_pachuca['mes'] - 1) // 3 + 1
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2932962646.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
8,9,1,Nueva,1506407.47,1
9,10,1,Nueva,989730.81,1
10,11,1,Nueva,631457.60,1
11,12,1,Nueva,495053.51,1
25,26,1,Nueva,588699.96,1
...,...,...,...,...,...
2472,2473,3,Usada,1027000.00,1
2473,2474,3,Usada,865429.51,1
2482,2483,3,Usada,996515.48,1
2487,2488,3,Usada,654787.62,1


In [195]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2023_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023
0,Nueva,"1,626,835.60"
1,Usada,"1,680,374.92"


In [196]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,626,835.60"
3,Usada,1T 2023,"1,680,374.92"


In [197]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        text=df_modalidad['promedio'],  # Texto con el valor de promedio
        textposition="top center",  # Posición del texto
    ))
    
fig.update_layout(
    title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2024_pueb', carpeta='graficas')

fig.show()

In [198]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff','#ff0059']
fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2024_pueb', carpeta='graficas')
fig.show()


In [199]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2024_pueb', carpeta='graficas')
fig.show()


### 2023

In [200]:
finance_2023_pachuca = df_financiamientos_2023_puebla[['id','mes','modalidad','monto']]
shape_2023 = finance_2023_pachuca.shape
print(shape_2023)
print(finance_2023_pachuca.head())

(10837, 4)
   id  mes  modalidad      monto
0   1    1          2   19817.75
1   2    1          2       0.00
2   3    1          2       0.00
3   4    1          2   33908.00
4   5    1          2  277298.00


In [201]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2023_pachuca['modalidad'] = finance_2023_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2023_pachuca.loc[~finance_2023_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2023_pachuca = finance_2023_pachuca[finance_2023_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2023_pachuca['trimestre'] = (finance_2023_pachuca['mes'] - 1) // 3 + 1
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2932962646.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
5,6,1,Nueva,1112324.29,1
7,8,1,Nueva,672522.08,1
8,9,1,Nueva,340303.84,1
12,13,1,Usada,1168245.82,1
13,14,1,Usada,481372.87,1
...,...,...,...,...,...
10817,10818,12,Usada,728810.59,4
10819,10820,12,Usada,461268.20,4
10823,10824,12,Usada,619000.00,4
10824,10825,12,Usada,800000.00,4


In [202]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2023_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023,4T 2023
0,Nueva,"1,617,135.57","1,721,878.13","1,701,100.83","1,709,858.12"
1,Usada,"1,388,331.92","1,455,266.44","1,511,701.71","1,615,482.02"


In [203]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,617,135.57"
4,Nueva,2T 2023,"1,721,878.13"
6,Nueva,3T 2023,"1,701,100.83"
8,Nueva,4T 2023,"1,709,858.12"
3,Usada,1T 2023,"1,388,331.92"
5,Usada,2T 2023,"1,455,266.44"
7,Usada,3T 2023,"1,511,701.71"
9,Usada,4T 2023,"1,615,482.02"


In [204]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        text=df_modalidad['promedio'],  # Texto con el valor de promedio
        textposition="top center",  # Posición del texto
    ))
    
fig.update_layout(
    title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pueb', carpeta='graficas')

fig.show()

In [205]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff','#ff0059']
fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pueb', carpeta='graficas')
fig.show()


In [206]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promXcredit_newold_2023_pueb', carpeta='graficas')
fig.show()


### 2022

In [207]:
finance_2022_pachuca = df_financiamientos_2022_puebla[['id','mes','modalidad','monto']]
shape_2022 = finance_2022_pachuca.shape
print(shape_2022)
print(finance_2022_pachuca.head())

(3104, 4)
   id  mes  modalidad     monto
0   1    1          2  21093.00
1   2    1          2  34318.00
2   3    1          2  43431.00
3   4    1          2  51998.00
4   5    1          2  94648.96


In [208]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022_pachuca['modalidad'] = finance_2022_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022_pachuca.loc[~finance_2022_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022_pachuca['trimestre'] = (finance_2022_pachuca['mes'] - 1) // 3 + 1
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\284171248.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
27,28,1,Nueva,148572.59,1
28,29,1,Nueva,243566.95,1
29,30,1,Usada,318287.55,1
30,31,1,Nueva,330190.00,1
31,32,1,Usada,385482.13,1
...,...,...,...,...,...
3081,3082,12,Usada,4378500.00,4
3082,3083,12,Usada,9600000.00,4
3084,3085,12,Usada,11916000.00,4
3085,3086,12,Nueva,17350000.00,4


In [209]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2022',
    2: '2T 2022',
    3: '3T 2022',
    4: '4T 2022'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2022,2T 2022,3T 2022,4T 2022
0,Nueva,"1,484,036.81","1,618,614.46","1,688,152.14","1,862,768.46"
1,Usada,"1,351,099.36","1,748,361.62","1,597,250.91","1,679,717.89"


In [210]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2022,"1,484,036.81"
4,Nueva,2T 2022,"1,618,614.46"
6,Nueva,3T 2022,"1,688,152.14"
8,Nueva,4T 2022,"1,862,768.46"
3,Usada,1T 2022,"1,351,099.36"
5,Usada,2T 2022,"1,748,361.62"
7,Usada,3T 2022,"1,597,250.91"
9,Usada,4T 2022,"1,679,717.89"


In [211]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]
colors = ['#9500ff', '#ff0059']

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2022)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2022_pueb', carpeta='graficas')
fig.show()


### 2021

In [212]:
finance_2021_pachuca = df_financiamientos_2021_puebla[['id','mes','modalidad','monto']]
shape_2021 = finance_2021_pachuca.shape
print(shape_2021)
print(finance_2021_pachuca.head())

(3481, 4)
   id  mes  modalidad      monto
0   1    1          1       0.00
1   2    1          1       0.00
2   3    1          1       0.00
3   4    1          1  602563.58
4   5    1          1  151281.71


In [213]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2021_pachuca['modalidad'] = finance_2021_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2021_pachuca.loc[~finance_2021_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2021_pachuca = finance_2021_pachuca[finance_2021_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2021_pachuca['trimestre'] = (finance_2021_pachuca['mes'] - 1) // 3 + 1
finance_2021_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\1979572604.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,0.00,1
2,3,1,Nueva,0.00,1
3,4,1,Nueva,602563.58,1
4,5,1,Nueva,151281.71,1
...,...,...,...,...,...
3471,3472,12,Usada,1095023.62,4
3472,3473,12,Usada,590858.29,4
3473,3474,12,Usada,795408.91,4
3474,3475,12,Usada,2562000.00,4


In [214]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_21 = finance_2021_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_21

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,477,735.94","1,812,868.35","1,669,859.52","1,755,915.42"
1,Usada,"1,484,791.50","1,535,121.46","1,404,221.52","1,488,837.86"


In [215]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_21 = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_21

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,477,735.94"
4,Nueva,2T 2021,"1,812,868.35"
6,Nueva,3T 2021,"1,669,859.52"
8,Nueva,4T 2021,"1,755,915.42"
3,Usada,1T 2021,"1,484,791.50"
5,Usada,2T 2021,"1,535,121.46"
7,Usada,3T 2021,"1,404,221.52"
9,Usada,4T 2021,"1,488,837.86"


In [216]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_21['trimestre']
modalidades = promedio_monto_por_modalidad_mes_21['modalidad']
promedios = promedio_monto_por_modalidad_mes_21['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        line=dict(color=colors[i % len(colors)]),  # Asignar colores a las líneas
        name=modalidad
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2021)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_21[promedio_monto_por_modalidad_mes_21['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2021_pueb', carpeta='graficas')
fig.show()


### 2020

In [217]:
finance_2020_pachuca = df_financiamientos_2020_puebla[['id','mes','modalidad','monto']]
shape_2020 = finance_2020_pachuca.shape
print(shape_2020)
print(finance_2020_pachuca.head())

(3192, 4)
   id  mes  modalidad       monto
0   1    1          1        0.00
1   2    1          1   369810.54
2   3    1          1   127585.00
3   4    1          1   453276.20
4   5    1          1  1472679.06


In [218]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2020_pachuca['modalidad'] = finance_2020_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2020_pachuca.loc[~finance_2020_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2020_pachuca = finance_2020_pachuca[finance_2020_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2020_pachuca['trimestre'] = (finance_2020_pachuca['mes'] - 1) // 3 + 1
finance_2020_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2090618855.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,369810.54,1
2,3,1,Nueva,127585.00,1
3,4,1,Nueva,453276.20,1
4,5,1,Nueva,1472679.06,1
...,...,...,...,...,...
3151,3152,12,Usada,703173.31,4
3152,3153,12,Usada,2066880.17,4
3153,3154,12,Usada,898000.00,4
3154,3155,12,Usada,1244623.98,4


In [219]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_20 = finance_2020_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_20

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,653,399.60","1,320,086.27","1,612,113.43","1,478,443.58"
1,Usada,"1,316,662.54","1,174,884.61","1,380,214.98","1,614,531.71"


In [220]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_20 = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_20

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,653,399.60"
4,Nueva,2T 2021,"1,320,086.27"
6,Nueva,3T 2021,"1,612,113.43"
8,Nueva,4T 2021,"1,478,443.58"
3,Usada,1T 2021,"1,316,662.54"
5,Usada,2T 2021,"1,174,884.61"
7,Usada,3T 2021,"1,380,214.98"
9,Usada,4T 2021,"1,614,531.71"


In [221]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_20['trimestre']
modalidades = promedio_monto_por_modalidad_mes_20['modalidad']
promedios = promedio_monto_por_modalidad_mes_20['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2020)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_20[promedio_monto_por_modalidad_mes_20['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2020_pueb', carpeta='graficas')
fig.show()


### 2019

In [222]:
finance_2019_pachuca = df_financiamientos_2019_puebla[['id','mes','modalidad','monto']]
shape_2019 = finance_2019_pachuca.shape
print(shape_2019)
print(finance_2019_pachuca.head())

(3448, 4)
   id  mes  modalidad  monto
0   1    1          1    0.0
1   2    1          1    0.0
2   3    1          1    0.0
3   4    1          1    0.0
4   5    1          1    0.0


In [223]:
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2019_pachuca['modalidad'] = finance_2019_pachuca['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2019_pachuca.loc[~finance_2019_pachuca['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2019_pachuca = finance_2019_pachuca[finance_2019_pachuca['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2019_pachuca['trimestre'] = (finance_2019_pachuca['mes'] - 1) // 3 + 1
finance_2019_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\2382405646.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,0.00,1
2,3,1,Nueva,0.00,1
3,4,1,Nueva,0.00,1
4,5,1,Nueva,0.00,1
...,...,...,...,...,...
3423,3424,12,Nueva,74486.38,4
3439,3440,12,Usada,13422.40,4
3440,3441,12,Usada,1228.87,4
3441,3442,12,Nueva,681926.15,4


In [224]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes_19 = finance_2019_pachuca.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.applymap('{:,.2f}'.format)
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes_19

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,826,202.71","1,757,781.43","1,535,964.72","1,502,314.18"
1,Usada,"1,198,518.92","1,325,789.05","1,387,760.80","1,384,186.31"


In [225]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes_19 = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['trimestre'] != 'index']
promedio_monto_por_modalidad_mes_19

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,826,202.71"
4,Nueva,2T 2021,"1,757,781.43"
6,Nueva,3T 2021,"1,535,964.72"
8,Nueva,4T 2021,"1,502,314.18"
3,Usada,1T 2021,"1,198,518.92"
5,Usada,2T 2021,"1,325,789.05"
7,Usada,3T 2021,"1,387,760.80"
9,Usada,4T 2021,"1,384,186.31"


In [226]:
# Datos
trimestres = promedio_monto_por_modalidad_mes_19['trimestre']
modalidades = promedio_monto_por_modalidad_mes_19['modalidad']
promedios = promedio_monto_por_modalidad_mes_19['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

fig = go.Figure()
# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2019)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto',gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes_19[promedio_monto_por_modalidad_mes_19['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_promcredit_newold_2019_pueb', carpeta='graficas')
fig.show()


___
___
# **5** *Estacionalidad*

## **Pachuca**

In [227]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','año','mes']]
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','año','mes']]

estacionalidad_2223_pachuca = pd.concat([finance_2022_pachuca, finance_2023_pachuca], ignore_index=True)
estacionalidad_2223_pachuca

,id,año,mes
0,1,2022,1
1,2,2022,1
2,3,2022,1
3,4,2022,1
4,5,2022,1
...,...,...,...
2064,677,2023,7
2065,678,2023,7
2066,679,2023,7
2067,680,2023,7


In [228]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
estacionalidad_2223_pachuca['trimestre'] = estacionalidad_2223_pachuca['mes'].apply(asignar_trimestre)
estacionalidad_2223_pachuca

,id,año,mes,trimestre
0,1,2022,1,1
1,2,2022,1,1
2,3,2022,1,1
3,4,2022,1,1
4,5,2022,1,1
...,...,...,...,...
2064,677,2023,7,3
2065,678,2023,7,3
2066,679,2023,7,3
2067,680,2023,7,3


In [229]:
conteo_trimestral = estacionalidad_2223_pachuca.groupby(['año', 'trimestre']).size().reset_index(name='conteo')
conteo_trimestral = conteo_trimestral[conteo_trimestral['año'] != 0]
# Concatenar 'año' y 'trimestre' en una nueva columna llamada 'trimestre'
conteo_trimestral['trimestre'] = conteo_trimestral['trimestre'].astype(str) + 'o ' + conteo_trimestral['año'].astype(str) 
# Eliminar la columna 'año' si ya no es necesaria
conteo_trimestral.drop(columns=['año'], inplace=True)
conteo_trimestral

,trimestre,conteo
1,1o 2022,329
2,2o 2022,323
3,3o 2022,341
4,4o 2022,395
5,1o 2023,267
6,2o 2023,310
7,3o 2023,103


In [230]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
import numpy as np
# Calcula el máximo valor de la serie y redondea hacia arriba al múltiplo más cercano de 50
max_y = np.ceil(conteo_trimestral['conteo'].max() / 50) * 50
# Crea un rango de valores de y en incrementos de 50 hasta el máximo
values_range = np.arange(0, max_y + 1, 50)
# Modifica los valores de 'values' en tu gráfico para que estén en incrementos de 50
values = conteo_trimestral['conteo'].apply(lambda x: min(values_range, key=lambda y: abs(y - x)))
# Luego, puedes proceder con la creación de tu gráfico como lo hiciste antes
labels = conteo_trimestral['trimestre']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,
        textposition='inside',
        showlegend=False,
        #width=0.5 
    )
)    
fig.update_layout(
    #title='Número de créditos por trimestre (2022-2023)',
    xaxis_title="Trimestre",        
    yaxis=dict(
        title="Créditos",
        gridcolor='#dddcda',
        tickvals=values_range,  # Especifica los valores de los ticks del eje y
        tickmode='array'  # Usa los valores especificados en 'tickvals' para los ticks
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_estacionalidad_Xcreditos_pachuca', carpeta='graficas')

fig.show()


## **Tulancingo**

In [231]:
finance_2022_pachuca = df_financiamientos_2022_tulancingo[['id','año','mes']]
finance_2023_pachuca = df_financiamientos_2023_tulancingo[['id','año','mes']]

estacionalidad_2223_pachuca = pd.concat([finance_2022_pachuca, finance_2023_pachuca], ignore_index=True)
estacionalidad_2223_pachuca

,id,año,mes
0,1,2022,1
1,2,2022,1
2,3,2022,1
3,4,2022,1
4,5,2022,1
...,...,...,...
356,41,2023,9
357,42,2023,9
358,43,2023,9
359,44,2023,9


In [232]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
estacionalidad_2223_pachuca['trimestre'] = estacionalidad_2223_pachuca['mes'].apply(asignar_trimestre)
estacionalidad_2223_pachuca

,id,año,mes,trimestre
0,1,2022,1,1
1,2,2022,1,1
2,3,2022,1,1
3,4,2022,1,1
4,5,2022,1,1
...,...,...,...,...
356,41,2023,9,3
357,42,2023,9,3
358,43,2023,9,3
359,44,2023,9,3


In [233]:
conteo_trimestral = estacionalidad_2223_pachuca.groupby(['año', 'trimestre']).size().reset_index(name='conteo')
conteo_trimestral = conteo_trimestral[conteo_trimestral['año'] != 0]
# Concatenar 'año' y 'trimestre' en una nueva columna llamada 'trimestre'
conteo_trimestral['trimestre'] = conteo_trimestral['trimestre'].astype(str) + 'o ' + conteo_trimestral['año'].astype(str) 
# Eliminar la columna 'año' si ya no es necesaria
conteo_trimestral.drop(columns=['año'], inplace=True)
conteo_trimestral

,trimestre,conteo
0,1o 2022,78
1,2o 2022,65
2,3o 2022,91
3,4o 2022,82
4,1o 2023,18
5,2o 2023,13
6,3o 2023,14


In [234]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
import numpy as np
# Calcula el máximo valor de la serie y redondea hacia arriba al múltiplo más cercano de 50
max_y = np.ceil(conteo_trimestral['conteo'].max() / 50) * 50
# Crea un rango de valores de y en incrementos de 50 hasta el máximo
values_range = np.arange(0, max_y + 1, 50)
# Modifica los valores de 'values' en tu gráfico para que estén en incrementos de 50
values = conteo_trimestral['conteo'].apply(lambda x: min(values_range, key=lambda y: abs(y - x)))
# Luego, puedes proceder con la creación de tu gráfico como lo hiciste antes
labels = conteo_trimestral['trimestre']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,
        textposition='inside',
        showlegend=False,
        #width=0.5 
    )
)    
fig.update_layout(
    #title='Número de créditos por trimestre (2022-2023)',
    xaxis_title="Trimestre",        
    yaxis=dict(
        title="Créditos",
        gridcolor='#dddcda',
        tickvals=values_range,  # Especifica los valores de los ticks del eje y
        tickmode='array'  # Usa los valores especificados en 'tickvals' para los ticks
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_estacionalidad_Xcreditos_tulgo', carpeta='graficas')

fig.show()


## **Puebla**

In [235]:
finance_2022_pachuca = df_financiamientos_2022_puebla[['id','año','mes']]
finance_2023_pachuca = df_financiamientos_2023_puebla[['id','año','mes']]
finance_2024_pachuca = df_financiamientos_2024_puebla[['id','año','mes']]


estacionalidad_2223_pachuca = pd.concat([finance_2022_pachuca, finance_2023_pachuca,finance_2024_pachuca], ignore_index=True)
estacionalidad_2223_pachuca

,id,año,mes
0,1,2022,1
1,2,2022,1
2,3,2022,1
3,4,2022,1
4,5,2022,1
...,...,...,...
16433,2493,2024,3
16434,2494,2024,3
16435,2495,2024,3
16436,2496,2024,3


In [236]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
estacionalidad_2223_pachuca['trimestre'] = estacionalidad_2223_pachuca['mes'].apply(asignar_trimestre)
estacionalidad_2223_pachuca

,id,año,mes,trimestre
0,1,2022,1,1
1,2,2022,1,1
2,3,2022,1,1
3,4,2022,1,1
4,5,2022,1,1
...,...,...,...,...
16433,2493,2024,3,1
16434,2494,2024,3,1
16435,2495,2024,3,1
16436,2496,2024,3,1


In [237]:
conteo_trimestral = estacionalidad_2223_pachuca.groupby(['año', 'trimestre']).size().reset_index(name='conteo')
conteo_trimestral = conteo_trimestral[conteo_trimestral['año'] != 0]
# Concatenar 'año' y 'trimestre' en una nueva columna llamada 'trimestre'
conteo_trimestral['trimestre'] = conteo_trimestral['trimestre'].astype(str) + 'o ' + conteo_trimestral['año'].astype(str) 
# Eliminar la columna 'año' si ya no es necesaria
conteo_trimestral.drop(columns=['año'], inplace=True)
conteo_trimestral

,trimestre,conteo
0,1o 2022,793
1,2o 2022,783
2,3o 2022,786
3,4o 2022,742
4,1o 2023,2441
5,2o 2023,2740
6,3o 2023,2618
7,4o 2023,3038
8,1o 2024,2497


In [242]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']
import numpy as np
# Calcula el máximo valor de la serie y redondea hacia arriba al múltiplo más cercano de 50
max_y = np.ceil(conteo_trimestral['conteo'].max() / 50) * 50
# Crea un rango de valores de y en incrementos de 50 hasta el máximo
values_range = np.arange(0, max_y + 1, 200)
# Modifica los valores de 'values' en tu gráfico para que estén en incrementos de 50
values = conteo_trimestral['conteo'].apply(lambda x: min(values_range, key=lambda y: abs(y - x)))
# Luego, puedes proceder con la creación de tu gráfico como lo hiciste antes
labels = conteo_trimestral['trimestre']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,
        textposition='inside',
        showlegend=False,
        #width=0.5 
    )
)    
fig.update_layout(
    #title='Número de créditos por trimestre (2022-2023)',
    xaxis_title="Trimestre",        
    yaxis=dict(
        title="Créditos",
        gridcolor='#dddcda',
        tickvals=values_range,  # Especifica los valores de los ticks del eje y
        tickmode='array'  # Usa los valores especificados en 'tickvals' para los ticks
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_estacionalidad_Xcreditos_puebla', carpeta='graficas')

fig.show()

___
___
# **6** *Créditos por porcentaje de UMAS*

## **Pachuca**

### 2022

In [243]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','ingresos_rango']]
registros_0 = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance_2022_pachuca.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

41
(1388, 2)


In [244]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,2.61-4.00
1,2,9.01-12.00
2,3,> 12.00
3,4,> 12.00
4,5,4.01-6.00
...,...,...
1383,1384,0
1384,1385,0
1385,1386,0
1386,1387,0


In [245]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2022_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2022_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,379,27.305476
1,6.01-9.00,210,15.129683
2,4.01-6.00,199,14.337176
3,≤ 2.60,199,14.337176
4,2.61-4.00,190,13.688761
5,9.01-12.00,170,12.247839


In [246]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
3,≤ 2.60,199,14.337176
4,2.61-4.00,190,13.688761
2,4.01-6.00,199,14.337176
1,6.01-9.00,210,15.129683
5,9.01-12.00,170,12.247839
0,> 12.00,379,27.305476


In [247]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2022_pachuca', carpeta='graficas')

fig.show()


### 2023

In [248]:
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','ingresos_rango']]
registros_0 = finance_2023_pachuca[finance_2023_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2023 = finance_2023_pachuca.shape
print(df_2023)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

30
(681, 2)


In [249]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,≤ 2.60
1,2,≤ 2.60
2,3,2.61-4.00
3,4,≤ 2.60
4,5,2.61-4.00
...,...,...
676,677,> 12.00
677,678,> 12.00
678,679,> 12.00
679,680,> 12.00


In [250]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2023_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2023_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,132,19.383260
1,6.01-9.00,113,16.593245
2,4.01-6.00,110,16.152717
3,2.61-4.00,109,16.005874
4,9.01-12.00,96,14.096916
5,≤ 2.60,91,13.362702


In [251]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,91,13.362702
3,2.61-4.00,109,16.005874
2,4.01-6.00,110,16.152717
1,6.01-9.00,113,16.593245
4,9.01-12.00,96,14.096916
0,> 12.00,132,19.383260


In [252]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2023_pachuca', carpeta='graficas')

fig.show()


## **Tulancningo**

### 2022

In [253]:
finance_2022_pachuca = df_financiamientos_2022_tulancingo[['id','ingresos_rango']]
registros_0 = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance_2022_pachuca.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

19
(316, 2)


In [254]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,2.61-4.00
1,2,0
2,3,≤ 2.60
3,4,9.01-12.00
4,5,2.61-4.00
...,...,...
311,312,> 12.00
312,313,6.01-9.00
313,314,> 12.00
314,315,> 12.00


In [255]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2022_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2022_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,2.61-4.00,70,22.151899
1,6.01-9.00,60,18.987342
2,> 12.00,60,18.987342
3,≤ 2.60,46,14.556962
4,4.01-6.00,38,12.025316
5,9.01-12.00,23,7.278481


In [256]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
3,≤ 2.60,46,14.556962
0,2.61-4.00,70,22.151899
4,4.01-6.00,38,12.025316
1,6.01-9.00,60,18.987342
5,9.01-12.00,23,7.278481
2,> 12.00,60,18.987342


In [257]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2022_tulgo', carpeta='graficas')

fig.show()


### 2023

In [258]:
finance_2023_pachuca = df_financiamientos_2023_tulancingo[['id','ingresos_rango']]
registros_0 = finance_2023_pachuca[finance_2023_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2023 = finance_2023_pachuca.shape
print(df_2023)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

0
(45, 2)


In [259]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,> 12.00
1,2,> 12.00
2,3,> 12.00
3,4,4.01-6.00
4,5,≤ 2.60
5,6,6.01-9.00
6,7,2.61-4.00
7,8,2.61-4.00
8,9,≤ 2.60
9,10,6.01-9.00


In [260]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2023_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2023_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,13,28.888889
1,9.01-12.00,8,17.777778
2,6.01-9.00,7,15.555556
3,2.61-4.00,7,15.555556
4,4.01-6.00,6,13.333333
5,≤ 2.60,4,8.888889


In [261]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,4,8.888889
3,2.61-4.00,7,15.555556
4,4.01-6.00,6,13.333333
2,6.01-9.00,7,15.555556
1,9.01-12.00,8,17.777778
0,> 12.00,13,28.888889


In [262]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2023_tulgo', carpeta='graficas')

fig.show()


## **Puebla**

### 2022

In [263]:
finance_2022_pachuca = df_financiamientos_2022_puebla[['id','ingresos_rango']]
registros_0 = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance_2022_pachuca.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

104
(3104, 2)


In [264]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2022_pachuca['ingresos_rango'] = finance_2022_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\184882518.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,2.61-4.00
1,2,2.61-4.00
2,3,2.61-4.00
3,4,4.01-6.00
4,5,≤ 2.60
...,...,...
3099,3100,4.01-6.00
3100,3101,2.61-4.00
3101,3102,≤ 2.60
3102,3103,≤ 2.60


In [265]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2022_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2022_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,610,19.652062
1,4.01-6.00,544,17.525773
2,2.61-4.00,526,16.945876
3,6.01-9.00,516,16.623711
4,≤ 2.60,427,13.756443
5,9.01-12.00,377,12.145619


In [266]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
4,≤ 2.60,427,13.756443
2,2.61-4.00,526,16.945876
1,4.01-6.00,544,17.525773
3,6.01-9.00,516,16.623711
5,9.01-12.00,377,12.145619
0,> 12.00,610,19.652062


In [267]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2022_puebla', carpeta='graficas')

fig.show()


### 2023

In [268]:
finance_2023_pachuca = df_financiamientos_2023_puebla[['id','ingresos_rango']]
registros_0 = finance_2023_pachuca[finance_2023_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2023 = finance_2023_pachuca.shape
print(df_2023)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

932
(10837, 2)


In [269]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,0
1,2,≤ 2.60
2,3,> 12.00
3,4,≤ 2.60
4,5,6.01-9.00
...,...,...
10832,10833,0
10833,10834,0
10834,10835,0
10835,10836,0


In [270]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2023_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2023_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,2067,19.073544
1,2.61-4.00,1812,16.720495
2,≤ 2.60,1730,15.963828
3,4.01-6.00,1602,14.782689
4,6.01-9.00,1578,14.561225
5,9.01-12.00,1116,10.298053


In [271]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
2,≤ 2.60,1730,15.963828
1,2.61-4.00,1812,16.720495
3,4.01-6.00,1602,14.782689
4,6.01-9.00,1578,14.561225
5,9.01-12.00,1116,10.298053
0,> 12.00,2067,19.073544


In [272]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2023_puebla', carpeta='graficas')

fig.show()


### 2024

In [273]:
finance_2023_pachuca = df_financiamientos_2024_puebla[['id','ingresos_rango']]
registros_0 = finance_2023_pachuca[finance_2023_pachuca['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2023 = finance_2023_pachuca.shape
print(df_2023)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

362
(2497, 2)


In [274]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
finance_2023_pachuca['ingresos_rango'] = finance_2023_pachuca['ingresos_rango'].replace({
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11468\4259768670.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,ingresos_rango
0,1,0
1,2,> 12.00
2,3,0
3,4,0
4,5,0
...,...,...
2492,2493,0
2493,2494,0
2494,2495,0
2495,2496,0


In [275]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance_2023_pachuca['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance_2023_pachuca)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,443,17.741290
1,2.61-4.00,414,16.579896
3,4.01-6.00,342,13.696436
4,≤ 2.60,339,13.576292
5,6.01-9.00,332,13.295955
6,9.01-12.00,265,10.612735


In [276]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
4,≤ 2.60,339,13.576292
1,2.61-4.00,414,16.579896
3,4.01-6.00,342,13.696436
5,6.01-9.00,332,13.295955
6,9.01-12.00,265,10.612735
0,> 12.00,443,17.741290


In [277]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_umas_2024_puebla', carpeta='graficas')

fig.show()


___
___
# **7** *Distribución % de créditos por organismo*

## **Pachuca**

### 2022

In [3]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','organismo']]
finance_2022_pachuca

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1383,1384,4
1384,1385,4
1385,1386,4
1386,1387,4


In [4]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -      41
1  BANJERCITO       7
2        CNBV     578
3      CONAVI       0
4    FOVISSTE     291
5   INFONAVIT     471
6       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].astype(str)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(categorias)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:16: SettingWithCopyWarning: 
A value is trying to

In [5]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,578,0.416427
1,INFONAVIT,471,0.339337
2,FOVISSTE,291,0.209654
3,-,41,0.029539
4,BANJERCITO,7,0.005043


In [6]:
f_2022 = finance_2022_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2776  registros en la base
Hay  1388  registros al contar


In [7]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\20988562.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)


,organismo,conteo,promedio
1,INFONAVIT,471,0.339337
0,CNBV,578,0.416427
2,FOVISSTE,291,0.209654
4,BANJERCITO,7,0.005043


In [8]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2022_pachuca', carpeta='graficas')

fig.show()

### 2023

In [9]:
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','organismo']]
finance_2023_pachuca

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
676,677,1
677,678,1
678,679,2
679,680,2


In [10]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -      11
1  BANJERCITO       2
2        CNBV     262
3      CONAVI       6
4    FOVISSTE     143
5   INFONAVIT     246
6       INSUS      11


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [11]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,262,0.384728
1,INFONAVIT,246,0.361233
2,FOVISSTE,143,0.209985
3,-,11,0.016153
4,INSUS,11,0.016153
5,CONAVI,6,0.008811
6,BANJERCITO,2,0.002937


In [12]:
f_2023 = finance_2023_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2023, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  1362  registros en la base
Hay  681  registros al contar


In [13]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'CONAVI': 4,
    'BANJERCITO': 5
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3316111741.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
1,INFONAVIT,246,0.361233
0,CNBV,262,0.384728
2,FOVISSTE,143,0.209985
5,CONAVI,6,0.008811
6,BANJERCITO,2,0.002937
4,INSUS,11,0.016153


In [14]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2023)',
    #title_font_size = 22,
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2023_pachuca', carpeta='graficas')

fig.show()

## **Tulancingo**

### 2022

In [15]:
finance_2022_pachuca = df_financiamientos_2022_tulancingo[['id','organismo']]
finance_2022_pachuca

,id,organismo
0,1,1
1,2,4
2,3,2
3,4,2
4,5,1
...,...,...
311,312,2
312,313,3
313,314,2
314,315,2


In [16]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

    organismo  conteo
0           -      19
1  BANJERCITO       1
2        CNBV      82
3      CONAVI       0
4    FOVISSTE      82
5   INFONAVIT     132
6       INSUS       0


In [17]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,132,0.417722
1,CNBV,82,0.259494
2,FOVISSTE,82,0.259494
3,-,19,0.060127
4,BANJERCITO,1,0.003165


In [18]:
f_2022 = finance_2022_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  632  registros en la base
Hay  316  registros al contar


In [19]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\20988562.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,132,0.417722
1,CNBV,82,0.259494
2,FOVISSTE,82,0.259494
4,BANJERCITO,1,0.003165


In [20]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2022_tulgo', carpeta='graficas')

fig.show()

### 2023

In [21]:
finance_2023_pachuca = df_financiamientos_2023_tulancingo[['id','organismo']]
finance_2023_pachuca

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,1
6,7,2
7,8,2
8,9,2
9,10,3


In [22]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       0
1        CNBV       9
2      CONAVI       0
3    FOVISSTE       9
4   INFONAVIT      27
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [23]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,27,0.6
1,CNBV,9,0.2
2,FOVISSTE,9,0.2


In [24]:
f_2023 = finance_2023_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2023, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  90  registros en la base
Hay  45  registros al contar


In [25]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'CONAVI': 4,
    'BANJERCITO': 5
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,27,0.6
1,CNBV,9,0.2
2,FOVISSTE,9,0.2


In [26]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2023)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2023_tulgo', carpeta='graficas')

fig.show()

## **Puebla**

### 2022

In [27]:
finance_2022_pachuca = df_financiamientos_2022_puebla[['id','organismo']]
finance_2022_pachuca

,id,organismo
0,1,2
1,2,2
2,3,2
3,4,2
4,5,1
...,...,...
3099,3100,1
3100,3101,1
3101,3102,1
3102,3103,1


In [28]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['organismo'] = finance_2022_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -      45
1  BANJERCITO      15
2        CNBV    1220
3      CONAVI      50
4    FOVISSTE     364
5   INFONAVIT    1391
6       INSUS      19


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3533323865.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [29]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2022_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,1391,0.448131
1,CNBV,1220,0.393041
2,FOVISSTE,364,0.117268
3,CONAVI,50,0.016108
4,-,45,0.014497
5,INSUS,19,0.006121
6,BANJERCITO,15,0.004832


In [30]:
f_2022 = finance_2022_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  6208  registros en la base
Hay  3104  registros al contar


In [31]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\20988562.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,1391,0.448131
1,CNBV,1220,0.393041
2,FOVISSTE,364,0.117268
6,BANJERCITO,15,0.004832
3,CONAVI,50,0.016108
5,INSUS,19,0.006121


In [32]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2022_puebla', carpeta='graficas')

fig.show()

### 2023

In [33]:
finance_2023_pachuca = df_financiamientos_2023_puebla[['id','organismo']]
finance_2023_pachuca

,id,organismo
0,1,4
1,2,1
2,3,1
3,4,2
4,5,2
...,...,...
10832,10833,1
10833,10834,4
10834,10835,4
10835,10836,4


In [34]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -     552
1  BANJERCITO      42
2        CNBV    3275
3      CONAVI     230
4    FOVISSTE     985
5   INFONAVIT    5654
6       INSUS      99


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [35]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5654,0.521731
1,CNBV,3275,0.302205
2,FOVISSTE,985,0.090892
3,-,552,0.050937
4,CONAVI,230,0.021224
5,INSUS,99,0.009135
6,BANJERCITO,42,0.003876


In [36]:
f_2023 = finance_2023_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2023, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  21674  registros en la base
Hay  10837  registros al contar


In [37]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'CONAVI': 4,
    'BANJERCITO': 5
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3316111741.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,5654,0.521731
1,CNBV,3275,0.302205
2,FOVISSTE,985,0.090892
4,CONAVI,230,0.021224
6,BANJERCITO,42,0.003876
5,INSUS,99,0.009135


In [38]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2023)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2023_puebla', carpeta='graficas')

fig.show()

### 2024

In [39]:
finance_2023_pachuca = df_financiamientos_2024_puebla[['id','organismo']]
finance_2023_pachuca

,id,organismo
0,1,4
1,2,2
2,3,4
3,4,4
4,5,4
...,...,...
2492,2493,4
2493,2494,4
2494,2495,4
2495,2496,4


In [40]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['organismo'] = finance_2023_pachuca['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        conteo_organismos = conteo_organismos.append({'organismo': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -     218
1  BANJERCITO       6
2        CNBV     660
3      CONAVI      18
4    FOVISSTE     187
5   INFONAVIT    1408
6       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\2208393616.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [41]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance_2023_pachuca['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,1408,0.563877
1,CNBV,660,0.264317
2,-,218,0.087305
3,FOVISSTE,187,0.074890
4,CONAVI,18,0.007209
5,BANJERCITO,6,0.002403


In [42]:
f_2023 = finance_2023_pachuca.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2023, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  4994  registros en la base
Hay  2497  registros al contar


In [43]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'CONAVI': 4,
    'BANJERCITO': 5
}

df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3316111741.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,1408,0.563877
1,CNBV,660,0.264317
3,FOVISSTE,187,0.074890
4,CONAVI,18,0.007209
5,BANJERCITO,6,0.002403


DEBO MANEJAR LO DE VARIABLES DONDE HAY 0% PARA QUE NO SE ELIMINE EN LAS GRÁFICAS

In [44]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2023)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_organismo_2024_puebla', carpeta='graficas')

fig.show()

___
___
# **8** *Distribución % de créditos por vivienda valor*

## **Pachuca**

### 2022

In [45]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','vivienda_valor']]
finance_2022_pachuca

,id,vivienda_valor
0,1,5
1,2,4
2,3,4
3,4,5
4,5,5
...,...,...
1383,1384,0
1384,1385,0
1385,1386,0
1386,1387,0


In [46]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,100
1,Económica,55
2,Media,530
3,Popular,79
4,Residencial,253
5,Residencial Plus,60
6,Tradicional,311


In [47]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,530,0.381844
1,Tradicional,311,0.224063
2,Residencial,253,0.182277
3,-,100,0.072046
4,Popular,79,0.056916
5,Residencial Plus,60,0.043228
6,Económica,55,0.039625


In [49]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

,vivienda_valor,conteo,promedio
6,Económica,55,0.039625
4,Popular,79,0.056916
1,Tradicional,311,0.224063
0,Media,530,0.381844
2,Residencial,253,0.182277
5,Residencial Plus,60,0.043228


In [50]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2022_pachuca', carpeta='graficas')

fig.show()

### 2023

In [51]:
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','vivienda_valor']]
finance_2023_pachuca

,id,vivienda_valor
0,1,2
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
676,677,4
677,678,4
678,679,6
679,680,5


In [52]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_viv = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_viv.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_viv['vivienda_valor'].unique():
        conteo_viv = conteo_viv.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_viv = conteo_viv.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_viv

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,74
1,Económica,47
2,Media,225
3,Popular,62
4,Residencial,83
5,Residencial Plus,24
6,Tradicional,166


In [53]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,225,0.330396
1,Tradicional,166,0.243759
2,Residencial,83,0.121880
3,-,74,0.108664
4,Popular,62,0.091043
5,Económica,47,0.069016
6,Residencial Plus,24,0.035242


In [54]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
5,Económica,47,0.069016
4,Popular,62,0.091043
1,Tradicional,166,0.243759
0,Media,225,0.330396
2,Residencial,83,0.121880
6,Residencial Plus,24,0.035242


In [56]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2023_pachuca', carpeta='graficas')

fig.show()

## **Tulancingo**

### 2022

In [57]:
finance_2022_pachuca = df_financiamientos_2022_tulancingo[['id','vivienda_valor']]
finance_2022_pachuca

,id,vivienda_valor
0,1,0
1,2,0
2,3,1
3,4,2
4,5,3
...,...,...
311,312,5
312,313,5
313,314,5
314,315,4


In [58]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,29
1,Económica,14
2,Media,101
3,Popular,38
4,Residencial,25
5,Residencial Plus,1
6,Tradicional,108


In [59]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Tradicional,108,0.341772
1,Media,101,0.319620
2,Popular,38,0.120253
3,-,29,0.091772
4,Residencial,25,0.079114
5,Económica,14,0.044304
6,Residencial Plus,1,0.003165


In [60]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
5,Económica,14,0.044304
2,Popular,38,0.120253
0,Tradicional,108,0.341772
1,Media,101,0.319620
4,Residencial,25,0.079114
6,Residencial Plus,1,0.003165


In [61]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2022_tulancingo', carpeta='graficas')

fig.show()

### 2023

In [62]:
finance_2023_pachuca = df_financiamientos_2023_tulancingo[['id','vivienda_valor']]
finance_2023_pachuca

,id,vivienda_valor
0,1,4
1,2,4
2,3,4
3,4,4
4,5,3
5,6,3
6,7,3
7,8,4
8,9,5
9,10,4


In [63]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_viv = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_viv.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_viv['vivienda_valor'].unique():
        conteo_viv = conteo_viv.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_viv = conteo_viv.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_viv

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,Económica,0
1,Media,26
2,Popular,5
3,Residencial,2
4,Residencial Plus,0
5,Tradicional,12


In [64]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,26,0.577778
1,Tradicional,12,0.266667
2,Popular,5,0.111111
3,Residencial,2,0.044444


In [65]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

,vivienda_valor,conteo,promedio
2,Popular,5,0.111111
1,Tradicional,12,0.266667
0,Media,26,0.577778
3,Residencial,2,0.044444


In [66]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2023_tulancingo', carpeta='graficas')

fig.show()

## **Puebla**

### 2022

In [67]:
finance_2022_pachuca = df_financiamientos_2022_puebla[['id','vivienda_valor']]
finance_2022_pachuca

,id,vivienda_valor
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
3099,3100,0
3100,3101,0
3101,3102,0
3102,3103,0


In [68]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2022_pachuca['vivienda_valor'] = finance_2022_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\3425943650.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,251
1,Económica,169
2,Media,897
3,Popular,447
4,Residencial,342
5,Residencial Plus,103
6,Tradicional,895


In [69]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2022_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2022_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,897,0.288982
1,Tradicional,895,0.288338
2,Popular,447,0.144008
3,Residencial,342,0.110180
4,-,251,0.080863
5,Económica,169,0.054446
6,Residencial Plus,103,0.033183


In [70]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
5,Económica,169,0.054446
2,Popular,447,0.144008
1,Tradicional,895,0.288338
0,Media,897,0.288982
3,Residencial,342,0.110180
6,Residencial Plus,103,0.033183


In [71]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2022_puebla', carpeta='graficas')
fig.show()

### 2023

In [72]:
finance_2023_pachuca = df_financiamientos_2023_puebla[['id','vivienda_valor']]
finance_2023_pachuca

,id,vivienda_valor
0,1,0
1,2,0
2,3,0
3,4,1
4,5,1
...,...,...
10832,10833,0
10833,10834,0
10834,10835,0
10835,10836,0


In [73]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_viv = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_viv.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_viv['vivienda_valor'].unique():
        conteo_viv = conteo_viv.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_viv = conteo_viv.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_viv

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,1938
1,Económica,343
2,Media,2865
3,Popular,1500
4,Residencial,1247
5,Residencial Plus,367
6,Tradicional,2577


In [74]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,2865,0.264372
1,Tradicional,2577,0.237796
2,-,1938,0.178832
3,Popular,1500,0.138415
4,Residencial,1247,0.115069
5,Residencial Plus,367,0.033865
6,Económica,343,0.031651


In [75]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
6,Económica,343,0.031651
3,Popular,1500,0.138415
1,Tradicional,2577,0.237796
0,Media,2865,0.264372
4,Residencial,1247,0.115069
5,Residencial Plus,367,0.033865


In [76]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2023_puebla', carpeta='graficas')

fig.show()

### 2024

In [77]:
finance_2023_pachuca = df_financiamientos_2024_puebla[['id','vivienda_valor']]
finance_2023_pachuca

,id,vivienda_valor
0,1,0
1,2,4
2,3,0
3,4,0
4,5,0
...,...,...
2492,2493,0
2493,2494,0
2494,2495,0
2495,2496,0


In [78]:
# Convertir la columna 'organismo' a cadena de texto (str)
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance_2023_pachuca['vivienda_valor'] = finance_2023_pachuca['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_viv = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
conteo_viv.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_viv['vivienda_valor'].unique():
        conteo_viv = conteo_viv.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_viv = conteo_viv.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_viv

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\1215214662.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,vivienda_valor,conteo
0,-,645
1,Económica,9
2,Media,666
3,Popular,283
4,Residencial,304
5,Residencial Plus,89
6,Tradicional,501


In [79]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance_2023_pachuca['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance_2023_pachuca)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,666,0.266720
1,-,645,0.258310
2,Tradicional,501,0.200641
3,Residencial,304,0.121746
4,Popular,283,0.113336
5,Residencial Plus,89,0.035643
6,Económica,9,0.003604


In [80]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13008\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
6,Económica,9,0.003604
4,Popular,283,0.113336
2,Tradicional,501,0.200641
0,Media,666,0.266720
3,Residencial,304,0.121746
5,Residencial Plus,89,0.035643


In [81]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2023)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_X_vivienda_2024_puebla', carpeta='graficas')

fig.show()